# Creación de Dataset

In [1]:
import os
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt

In [2]:
# Diccionario con los cambios de mes (str a número)
cambio_mes = {'Jan': '01', 'Feb': '02', 'Mar': '03', 'Apr': '04',
              'May': '05', 'Jun': '06', 'Jul': '07', 'Aug': '08',
              'Sep': '09', 'Oct': '10', 'Nov': '11', 'Dec': '12'
              }

## 1. Dataset de Desempleo por Grupos

In [3]:
# Cargando CSV
unemployemnt_data_usa = pd.read_csv('./Data/Raw/Unemployment_Data_us.csv')
# Día genérico
unemployemnt_data_usa['Day'] = 1

# Obteniendo el número de mes en vez del str
unemployemnt_data_usa['Month'] = unemployemnt_data_usa['Month'].apply(lambda x: cambio_mes.get(x))

# Ordenando el dataset y reiniciando el índice
unemployemnt_data_usa = unemployemnt_data_usa.sort_values(['Year', 'Month']).reset_index(drop=True)

# Obteniendo la fecha
unemployemnt_data_usa[['Year', 'Month', 'Day']] = unemployemnt_data_usa[['Year', 'Month', 'Day']].astype(str)
unemployemnt_data_usa['Date'] = unemployemnt_data_usa[['Year', 'Month', 'Day']].agg('-'.join, axis=1)
unemployemnt_data_usa['Date'] = unemployemnt_data_usa['Date'].apply(lambda x: datetime.strptime(x, "%Y-%m-%d"))

# Seleccionando las columnas que necesitamos
unemployemnt_data_usa = unemployemnt_data_usa[['Date', 'Primary_School', 'High_School', 'Associates_Degree', 'Professional_Degree',
                                               'White', 'Black', 'Asian', 'Hispanic', 'Men', 'Women']]

# Cambiando los nombres de las columnas
unemployemnt_data_usa.columns = ['Fecha', 'Escuela_Primaria', 'Preparatoria', 'Tecnico', 'Profesional',
                                 'Blanco', 'Negro', 'Asiatico', 'Hispano', 'Hombre', 'Mujer']

# Quitando el año 2020 porque tienen NaN
unemployemnt_data_usa = unemployemnt_data_usa[unemployemnt_data_usa['Fecha'] < datetime(2020, 1, 1)]

# Exportando
unemployemnt_data_usa.to_excel(r'./Data/Procesado/Desempleo_Por_Grupo.xlsx', index=False)

unemployemnt_data_usa.head()

,Fecha,Escuela_Primaria,Preparatoria,Tecnico,Profesional,Blanco,Negro,Asiatico,Hispano,Hombre,Mujer
0,2010-01-01,15.3,10.2,8.6,4.9,8.8,16.5,8.3,12.9,10.2,7.9
1,2010-02-01,15.8,10.7,8.1,4.9,8.9,16.1,8.2,12.7,10.3,8.0
2,2010-03-01,14.9,11.0,8.4,4.9,8.9,16.8,7.6,12.9,10.2,8.1
3,2010-04-01,14.7,10.8,8.4,4.8,9.0,16.6,7.0,12.5,10.2,8.3
4,2010-05-01,14.6,10.9,8.2,4.6,8.7,15.5,7.8,12.0,9.7,8.1


## 2. Generando Archivo Poblacional por Raza

In [4]:
# Path
path = './Data/Raw/Population_States_Race/'

# Df que contendrá los datos de los estados
df = pd.DataFrame()

# Ciclo para crear un archivo parquet con la concatenación de los csv
for file in os.listdir(path):
    # Solo los archivos .csv
    if file.endswith('.csv'):
        # Obteniendo el año del archivo
        anio = '20' + file.split('.')[0].split('_')[-1]
        # Cargando el archivo
        temp = pd.read_csv(path + file)
        # Agregando el año a una columna
        temp['Year'] = anio
        # Concatenando todos los archivos
        df = pd.concat([df, temp], axis=0)

# Eliminando columnas que no usaremos
df.drop(['American Indian/Alaska Native', 'Native Hawaiian/Other Pacific Islander', 'Multiple Races', 'Footnotes'], axis=1, inplace=True)
# Ordenando las columnas
df = df[['Location', 'Year', 'White', 'Black', 'Asian', 'Hispanic']]
# Renombrando las columnas
df.columns = ['Estado', 'Year', 'Blanco_Cantidad', 'Negro_Cantidad', 'Asiatico_Cantidad', 'Hispano_Cantidad']
# Quitando 'United States' porque no es un estado y Puerto Rico
df_estados = df[~df['Estado'].isin(['United States', 'Puerto Rico'])].reset_index(drop=True)
# Tomando solo 'United States' porque es el total de la población
df_total = df[df['Estado'] == 'United States'].reset_index(drop=True)

## 3) Generando Archivo Poblacional por Género

In [5]:
# Path
path = './Data/Raw/Population_States_Gender/'

# Df que contendrá los datos de los estados
df_gender = pd.DataFrame()

# Ciclo para crear un archivo parquet con la concatenación de los csv
for file in os.listdir(path):
    # Solo los archivos .csv
    if file.endswith('.csv'):
        # Obteniendo el año del archivo
        anio = '20' + file.split('.')[0].split('_')[-1]
        # Cargando el archivo
        temp = pd.read_csv(path + file)
        # Agregando el año a una columna
        temp['Year'] = anio
        # Concatenando todos los archivos
        df_gender = pd.concat([df_gender, temp], axis=0)

# Eliminando columnas que no usaremos
#df.drop(['Footnotes'], axis=1, inplace=True)
# Ordenando las columnas
df_gender = df_gender[['Location', 'Year', 'Male', 'Female']]
# Renombrando las columnas
df_gender.columns = ['Estado', 'Year', 'Hombre_Cantidad', 'Mujer_Cantidad']
# Quitando 'United States' porque no es un estado y Puerto Rico
df_estados_gender = df_gender[~df_gender['Estado'].isin(['United States', 'Puerto Rico'])].reset_index(drop=True)
# Tomando solo 'United States' porque es el total de la población
df_total_gender = df_gender[df_gender['Estado'] == 'United States'].reset_index(drop=True)

## 4) Juntando y Exportando

In [6]:
df_estados_gen = df_estados.merge(df_estados_gender, on=['Estado', 'Year'], how='inner')

df_total_gen = df_total.merge(df_total_gender, on=['Estado', 'Year'], how='inner')

In [7]:
df_estados_gen.head()

,Estado,Year,Blanco_Cantidad,Negro_Cantidad,Asiatico_Cantidad,Hispano_Cantidad,Hombre_Cantidad,Mujer_Cantidad
0,Alabama,2010,3133000.0,1212100.0,49300.0,176400,2243900,2407500
1,Alaska,2010,439300.0,20500.0,35500.0,37900,347700,336900
2,Arizona,2010,3628600.0,236200.0,165900.0,1857400,3077700,3179300
3,Arkansas,2010,2113800.0,443000.0,32300.0,178300,1381900,1452100
4,California,2010,14600800.0,2042700.0,4777200.0,13825100,17997100,18518600


In [8]:
df_total_gen.head()

,Estado,Year,Blanco_Cantidad,Negro_Cantidad,Asiatico_Cantidad,Hispano_Cantidad,Hombre_Cantidad,Mujer_Cantidad
0,United States,2010,191844300.0,36217100.0,14275400.0,49603000,146735000,154071300
1,United States,2011,192080600.0,36478000.0,14557700.0,50801600,147958500,155122200
2,United States,2012,192305000.0,36797500.0,15067900.0,51773400,149124800,156251800
3,United States,2013,192379500.0,37122600.0,15505700.0,52764900,150298300,157207200
4,United States,2014,192419600.0,37599400.0,16156500.0,54059600,151538800,158673400


In [9]:
# Exportando
df_estados_gen.to_excel('./Data/Procesado/Poblacion_Estados.xlsx', index=False)
df_total_gen.to_excel('./Data/Procesado/Poblacion_Total.xlsx', index=False)

## 5) Agregando las Magnitudes

In [10]:
# Cargando Dataset
df_estados_gen = pd.read_excel('./Data/Procesado/Poblacion_Estados.xlsx')
df_estados_gen.head()

,Estado,Year,Blanco_Cantidad,Negro_Cantidad,Asiatico_Cantidad,Hispano_Cantidad,Hombre_Cantidad,Mujer_Cantidad
0,Alabama,2010,3133000,1212100.0,49300,176400,2243900,2407500
1,Alaska,2010,439300,20500.0,35500,37900,347700,336900
2,Arizona,2010,3628600,236200.0,165900,1857400,3077700,3179300
3,Arkansas,2010,2113800,443000.0,32300,178300,1381900,1452100
4,California,2010,14600800,2042700.0,4777200,13825100,17997100,18518600


In [11]:
# Cargando Dataset de los estados
df_total = pd.read_excel('./Data/Procesado/Poblacion_Total.xlsx')

df_total.drop(columns=['Estado'], inplace=True)

df_total.head()

,Year,Blanco_Cantidad,Negro_Cantidad,Asiatico_Cantidad,Hispano_Cantidad,Hombre_Cantidad,Mujer_Cantidad
0,2010,191844300,36217100,14275400,49603000,146735000,154071300
1,2011,192080600,36478000,14557700,50801600,147958500,155122200
2,2012,192305000,36797500,15067900,51773400,149124800,156251800
3,2013,192379500,37122600,15505700,52764900,150298300,157207200
4,2014,192419600,37599400,16156500,54059600,151538800,158673400


In [12]:
# Cargando Dataset General
desempleo = pd.read_excel('./Data/Procesado/Desempleo_Por_Grupo.xlsx')

# Sacando porcentaje para las columnas[ Blanco, Negro, Asiatico, Hispano, Hombre, Mujer] con apply
desempleo[['Blanco', 'Negro', 'Asiatico', 'Hispano', 'Hombre', 'Mujer']] = desempleo[['Blanco', 'Negro', 'Asiatico', 'Hispano', 'Hombre', 'Mujer']].apply(lambda x: x / 100)

# Sacando el año de la fecha
desempleo['Year'] = desempleo['Fecha'].apply(lambda x: x.year)

# Seleccionando columnas
desempleo = desempleo[['Fecha', 'Year', 'Blanco', 'Negro', 'Asiatico', 'Hispano', 'Hombre', 'Mujer']]

desempleo.head()

,Fecha,Year,Blanco,Negro,Asiatico,Hispano,Hombre,Mujer
0,2010-01-01,2010,0.088,0.165,0.083,0.129,0.102,0.079
1,2010-02-01,2010,0.089,0.161,0.082,0.127,0.103,0.080
2,2010-03-01,2010,0.089,0.168,0.076,0.129,0.102,0.081
3,2010-04-01,2010,0.090,0.166,0.070,0.125,0.102,0.083
4,2010-05-01,2010,0.087,0.155,0.078,0.120,0.097,0.081


In [13]:
# DF general
df_general = desempleo.merge(df_total, on=['Year'], how='inner')

# Multiplicaciones porcentual por valor
df_general['Blanco_Cantidad'] = (df_general['Blanco'] * df_general['Blanco_Cantidad']).astype(int)
df_general['Negro_Cantidad'] = (df_general['Negro'] * df_general['Negro_Cantidad']).astype(int)
df_general['Asiatico_Cantidad'] = (df_general['Asiatico'] * df_general['Asiatico_Cantidad']).astype(int)
df_general['Hispano_Cantidad'] = (df_general['Hispano'] * df_general['Hispano_Cantidad']).astype(int)
df_general['Hombre_Cantidad'] = (df_general['Hombre'] * df_general['Hombre_Cantidad']).astype(int)
df_general['Mujer_Cantidad'] = (df_general['Mujer'] * df_general['Mujer_Cantidad']).astype(int)

# Multiplicando por 100 para obtener el porcentaje
df_general['Blanco'] = df_general['Blanco'] * 100
df_general['Negro'] = df_general['Negro'] * 100
df_general['Asiatico'] = df_general['Asiatico'] * 100
df_general['Hispano'] = df_general['Hispano'] * 100
df_general['Hombre'] = df_general['Hombre'] * 100
df_general['Mujer'] = df_general['Mujer'] * 100

# Exportando
df_general.to_excel('./Data/Procesado/Desempleo_USA_General.xlsx', index=False)

df_general.head()

,Fecha,Year,Blanco,Negro,Asiatico,Hispano,Hombre,Mujer,Blanco_Cantidad,Negro_Cantidad,Asiatico_Cantidad,Hispano_Cantidad,Hombre_Cantidad,Mujer_Cantidad
0,2010-01-01,2010,8.8,16.5,8.3,12.9,10.2,7.9,16882298,5975821,1184858,6398787,14966969,12171632
1,2010-02-01,2010,8.9,16.1,8.2,12.7,10.3,8.0,17074142,5830953,1170582,6299581,15113705,12325704
2,2010-03-01,2010,8.9,16.8,7.6,12.9,10.2,8.1,17074142,6084472,1084930,6398787,14966969,12479775
3,2010-04-01,2010,9.0,16.6,7.0,12.5,10.2,8.3,17265987,6012038,999278,6200375,14966969,12787917
4,2010-05-01,2010,8.7,15.5,7.8,12.0,9.7,8.1,16690454,5613650,1113481,5952360,14233294,12479775
